In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
import os
import copy

In [2]:
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': []
    }

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # print(f"y_train shape = {y_train.shape}, y_test shape = {y_test.shape}")
        history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_test = np.argmax(y_test, axis=1) 
        
        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['precision'].append(precision_score(y_test, y_pred, average='macro'))
        scores['recall'].append(recall_score(y_test, y_pred, average='macro'))
        scores['f1'].append(f1_score(y_test, y_pred, average='macro'))
        scores['roc_auc'].append(roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro'))  # ROC AUC 多分类问题

    avg_scores = {key: np.mean(value) for key, value in scores.items()}
    return avg_scores

In [3]:
def load_cifar10_data(data_dir):
    X = []
    Y = []

    # Load training data batches
    for i in range(1, 6):
        with open(os.path.join(data_dir, f'data_batch_{i}'), 'rb') as file:
            batch = pickle.load(file, encoding='latin1')
            X.append(batch['data'])
            Y.extend(batch['labels'])

    # Combine training data
    X = np.concatenate(X)

    # Load test data
    with open(os.path.join(data_dir, 'test_batch'), 'rb') as file:
        test_batch = pickle.load(file, encoding='latin1')
        X_test = test_batch['data']
        Y_test = test_batch['labels']

    # Combine test data
    X = np.vstack((X, X_test))
    Y.extend(Y_test)

    # Convert labels to numpy array
    Y = np.array(Y)

    # Note: Removing the reshaping operation
    # X = X.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (N, H, W, C)

    # Load label names from batches.meta
    with open(os.path.join(data_dir, 'batches.meta'), 'rb') as file:
        meta = pickle.load(file, encoding='latin1')
        label_names = meta['label_names']

    return X, Y, label_names


In [4]:
data_dir = 'cifar-10-batches-py'  # path directory
X_combined, y_combined, meta = load_cifar10_data(data_dir)

In [5]:
def build_simple_cnn(input_shape, num_classes, learning_rate=0.0005, num_filter=16, filter_size=(3, 3)):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  
    model.add(layers.Conv2D(num_filter, filter_size, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
print(y_combined.shape)

(60000,)


In [7]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters = [8]  
learning_rates = [0.001, 0.0005, 0.0015, 0.01, 0.005]  
filter_sizes = [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]  

highest_config = []
highest_accuracy = 0

for num_filter in num_filters:
    for learning_rate in learning_rates:  
        highest_filter_size = (0, 0)
        highest_filter_size_accuracy = 0
        best_filter_size = False
        
        for filter_size in filter_sizes:  
            current_config = [learning_rate, num_filter, filter_size]
            model = build_simple_cnn(input_shape, num_classes, learning_rate, num_filter, filter_size)

            X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
            y_combined_categorical = to_categorical(y_combined, num_classes=10)
            X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

            with tf.device('/GPU:0'):
                avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)

            print("current_config: ", current_config)
            print(avg_scores)

            if avg_scores['accuracy'] >= highest_filter_size_accuracy:
                highest_filter_size_accuracy = avg_scores['accuracy']
                highest_filter_size = current_config[2]
                best_filter_size = False
            elif highest_filter_size_accuracy > avg_scores['accuracy'] and filter_size > highest_filter_size:
                print("Last filter_size is best, break")
                best_filter_size = True

            if avg_scores['accuracy'] > highest_accuracy:
                highest_config = current_config
                highest_accuracy = avg_scores['accuracy']
                print("highest_config: ", highest_config)
                print("highest_accuracy: ", highest_accuracy)

            if best_filter_size:
                break

print("highest_config: ", highest_config)
print("highest_accuracy: ", highest_accuracy)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4029 - loss: 1.7078 - val_accuracy: 0.3926 - val_loss: 1.7451
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5963 - loss: 1.1557 - val_accuracy: 0.5677 - val_loss: 1.2520
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6606 - loss: 0.9622 - val_accuracy: 0.5717 - val_loss: 1.2530
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7130 - loss: 0.8282 - val_accuracy: 0.5793 - val_loss: 1.2521
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7571 - loss: 0.7110 - val_accuracy: 0.5789 - val_loss: 1.3011
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7322 - loss: 0.8015 - val_accuracy: 0.7424 - val_loss: 0.7263
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7841 - loss: 0.6473 - val_accuracy: 0.7400 - val_loss: 0.7289
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.818

In [24]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters=[8, 16, 32, 64, 128]
best_learning_rate=highest_config[0]
best_filter_sizes=highest_config[2]

highest_config_last=[]
highest_accuracy_last=0
for num_filter in num_filters:
    current_config = [best_learning_rate, num_filter, best_filter_sizes]
    model = build_simple_cnn(input_shape, num_classes, best_learning_rate, num_filter, best_filter_sizes)

    X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
    y_combined_categorical = to_categorical(y_combined, num_classes=10)
    X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0
    with tf.device('/GPU:0'):
        avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)
    print("current_config: ", current_config)
    print(avg_scores)

    if avg_scores['accuracy'] > highest_accuracy_last:
        highest_config_last = current_config
        highest_accuracy_last = avg_scores['accuracy']
        print("highest_config_last: ", highest_config_last)
        print("highest_accuracy_last: ", highest_accuracy_last)
            
print("highest_config_last: ", highest_config_last)
print("highest_accuracy_last: ", highest_accuracy_last)

Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3974 - loss: 1.7300 - val_accuracy: 0.4892 - val_loss: 1.5038
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5679 - loss: 1.2266 - val_accuracy: 0.5513 - val_loss: 1.2709
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6320 - loss: 1.0647 - val_accuracy: 0.5777 - val_loss: 1.2249
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6773 - loss: 0.9369 - val_accuracy: 0.5882 - val_loss: 1.2157
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7169 - loss: 0.8293 - val_accuracy: 0.5798 - val_loss: 1.2520
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7081 - loss: 0.8582 - val_accuracy: 0.7293 - val_loss: 0.7811
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7500 - loss: 0.7512 - val_accuracy: 0.7023 - val_loss: 0.8444
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7809 